XGBOOST의 hyperparameter 목록은 다음과 같습니다.

https://xgboost.readthedocs.io/en/stable/parameter.html

보스턴 집값 데이터를 이용할 겁니다.

GridSearch로 가장 좋은 hyperparameter set을 찾아서 집값을 가장 잘 예측하는 xgboost 모델을 만들어봅시다.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.impute import SimpleImputer

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
df = pd.read_csv('train.csv')

In [7]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [8]:
df[['YearBuilt', 'LotFrontage', 'LotArea', 'SalePrice']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   YearBuilt    1460 non-null   int64  
 1   LotFrontage  1201 non-null   float64
 2   LotArea      1460 non-null   int64  
 3   SalePrice    1460 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 45.8 KB


In [9]:
df[['YearBuilt', 'LotFrontage', 'LotArea', 'SalePrice']]

,YearBuilt,LotFrontage,LotArea,SalePrice
0,2003,65.0,8450,208500
1,1976,80.0,9600,181500
2,2001,68.0,11250,223500
3,1915,60.0,9550,140000
4,2000,84.0,14260,250000
...,...,...,...,...
1455,1999,62.0,7917,175000
1456,1978,85.0,13175,210000
1457,1941,66.0,9042,266500
1458,1950,68.0,9717,142125


In [10]:
df[['YearBuilt', 'LotFrontage', 'LotArea', 'SalePrice']].dropna()

,YearBuilt,LotFrontage,LotArea,SalePrice
0,2003,65.0,8450,208500
1,1976,80.0,9600,181500
2,2001,68.0,11250,223500
3,1915,60.0,9550,140000
4,2000,84.0,14260,250000
...,...,...,...,...
1455,1999,62.0,7917,175000
1456,1978,85.0,13175,210000
1457,1941,66.0,9042,266500
1458,1950,68.0,9717,142125


In [11]:
df_new = df[['YearBuilt', 'LotFrontage', 'LotArea', 'SalePrice']].dropna()

In [12]:
features = df_new.iloc[:, :-1]

In [13]:
len(features)

1201

In [14]:
features

,YearBuilt,LotFrontage,LotArea
0,2003,65.0,8450
1,1976,80.0,9600
2,2001,68.0,11250
3,1915,60.0,9550
4,2000,84.0,14260
...,...,...,...
1455,1999,62.0,7917
1456,1978,85.0,13175
1457,1941,66.0,9042
1458,1950,68.0,9717


In [15]:
label = df_new.SalePrice

In [16]:
label

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1201, dtype: int64

In [17]:
X = df_new.iloc[:, :-1].values
y = df_new.iloc[:, -1].values

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
X

array([[ 2003.,    65.,  8450.],
       [ 1976.,    80.,  9600.],
       [ 2001.,    68., 11250.],
       ...,
       [ 1941.,    66.,  9042.],
       [ 1950.,    68.,  9717.],
       [ 1965.,    75.,  9937.]])

In [20]:
y

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
X_test

array([[-1.08647363e+00, -8.01119812e-01, -4.69055153e-01],
       [-7.06158338e-01, -3.92037355e-01,  2.04208641e-01],
       [-5.16000693e-01,  3.03402822e-01, -1.91500252e-01],
       [ 1.06864635e+00, -3.92037355e-01,  6.46394608e-01],
       [ 1.00526047e+00,  2.21586331e-01, -2.42541083e-02],
       [ 2.76322828e-01, -2.28404372e-01, -3.48048349e-01],
       [ 8.15102823e-01,  9.17026508e-01, -3.09574752e-03],
       [-4.20921871e-01, -9.23844549e-01,  7.57773170e-01],
       [ 2.44629887e-01,  1.80678085e-01, -2.49864045e-01],
       [ 8.78488704e-01, -2.69312618e-01, -1.02468723e-01],
       [ 2.27793012e-02, -1.98745894e+00, -9.82561438e-01],
       [-6.74465398e-01,  5.07944051e-01, -4.36961011e-01],
       [ 9.10181645e-01, -1.86473420e+00, -8.67854594e-01],
       [ 1.22711105e+00,  9.88615938e-02, -1.55245758e-01],
       [ 1.06864635e+00,  1.57155844e+00,  4.12701702e-01],
       [-1.99071285e-01, -1.46587880e-01, -2.55094202e-01],
       [ 8.78488704e-01,  1.80678085e-01

In [23]:
from xgboost import XGBRegressor

In [24]:
xgb = XGBRegressor()

In [25]:
xgb

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None,
             reg_alpha=None, reg_lambda=None, ...)

In [26]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [27]:
y_pred = xgb.predict(X_test)

In [28]:
y_pred

array([100186.37 , 115476.76 , 131348.92 , 237449.25 , 238680.6  ,
       138820.33 , 287776.97 , 144184.14 , 129797.66 , 205694.44 ,
        91207.71 , 129345.7  , 175670.16 , 221427.56 , 359402.62 ,
       134089.92 , 223629.44 , 358084.12 , 127940.97 , 281604.84 ,
       136811.64 , 355759.44 , 139492.12 , 164526.34 , 121502.04 ,
       406956.75 , 273610.1  , 115114.31 , 175670.16 , 238692.4  ,
       128283.04 , 170773.2  , 108747.48 , 181601.22 , 336931.56 ,
       182415.3  , 151239.03 , 115290.58 , 132993.38 , 118782.336,
       146268.34 , 172544.33 , 212820.86 , 161431.88 , 179360.2  ,
       150960.45 , 101127.414, 127107.9  , 131497.83 , 186991.06 ,
       142528.34 , 127077.34 , 243780.17 , 422621.1  , 193667.66 ,
       135044.81 , 200228.81 , 366983.34 , 147809.17 , 120214.766,
        99966.625, 265297.06 , 112005.625, 180680.1  , 100126.7  ,
       163424.2  , 268732.94 , 136023.33 , 203659.31 , 209604.4  ,
       156283.98 , 335675.62 , 297665.1  ,  93609.04 , 206426.

In [29]:
from xgboost import XGBRegressor

In [30]:
xgb = XGBRegressor()

In [31]:
from sklearn.model_selection import KFold, cross_val_score

In [32]:
kfold = KFold(n_splits=5)

In [33]:
param_grid = {
    "eta" : [0.1, 0.3, 0.7],
    "gamma" : [0,1,2,3],
    "max_depth" : [None, 6, 9, 12],
    "max_features" : ['auto', 'sqrt']
}

In [34]:
from xgboost import XGBClassifier

In [35]:
xgb2 = XGBClassifier()

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
kfold2 = KFold(n_splits=10)

In [38]:
grid_search = GridSearchCV(estimator = xgb,
                          param_grid = param_grid,
                          cv = kfold2)

In [39]:
grid_search.fit(features, label)

[01:26:15] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:16] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:16] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  Thi

[01:26:21] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:21] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:21] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  Thi

[01:26:26] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:26] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:26] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  Thi

[01:26:33] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:34] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:34] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  Thi

[01:26:44] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:44] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:26:44] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  Thi

...

[01:31:42] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:31:42] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:31:42] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  Thi

[01:31:47] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:31:47] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:31:48] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  Thi

[01:31:55] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:31:55] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:31:56] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  Thi

[01:32:05] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constra...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                          

In [40]:
grid_search.best_params_

{'eta': 0.1, 'gamma': 0, 'max_depth': None, 'max_features': 'auto'}

In [41]:
xgb_best = XGBRegressor(eta = 0.1,
                        max_depth = None,
                        gamma = 0,
                        max_features = 'auto')

In [42]:
xgb_default_hyp = XGBRegressor()

In [43]:
xgb_default_hyp.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [44]:
y_pred_default_hyp = xgb_default_hyp.predict(X_test)

In [45]:
from sklearn import metrics

In [46]:
y_true = y_pred

In [47]:
y_pred = y_pred_default_hyp

In [49]:
metrics.r2_score(y_true, y_pred)

1.0